CódigraphTesto disponível [na minha página do Github](https://github.com/arthurkenzo/atividades_ia525)

### Importando bibliotecas

In [ ]:
import cvxpy as cp
import numpy as np
import mosek
import matplotlib.pyplot as plt
import time
import networkx as nx

from typing import Tuple

## Questão 1: Pintar o tabuleiro

### Modelo

## Questão 2: Sudoku

### Modelo:

\begin{align*}
    \text{Minimizar }   & ijk x_{ijk}\\
    \text{sujeito a }   & \sum_{i} x_{ijk} = 1 \quad \forall  j,k  \hspace{20pt} \text{(Unicidade de cada número em cada coluna)} \\
                        & \sum_{j} x_{ijk} = 1 \quad \forall  i,k  \hspace{20pt} \text{(Unicidade de cada número em cada linha)} \\
                        & \sum_{p,p',q,q'} x_{(3p+q),(3p'+q'),k} = 1 \quad \forall  k;  \quad p,p',q,q' \in \{0,1,2\} \hspace{20pt} \text{(Unicidade de cada número em cada submatriz)} \\
                        & x_{ijk}\in \mathbb{B} \quad \forall  i,j,k  \hspace{20pt} \text{(Variáveis de decisão binárias)} \\
\end{align*}

onde os índices $(i,j,k)$ que indexam as variáveis são números interios no intervalo $[0,8]$, e as variáveis de decisão $x_{ijk} \in \mathbb{B}$ são não nulas $\iff$ $k$ for o número escolhido para a casa $(i,j)$ do tabuleiro.  Como o problema pode ser altamente degenerado (por se tratar de um problema de factibilidade), definimos um ordenamento arbitrário das soluções através da função objetivo.

In [ ]:
def SolveSudoku(board:np.ndarray) -> np.ndarray:

    boardSize = 9

    decisionVars = np.empty((boardSize, boardSize, boardSize), dtype=object)
    for i in range(boardSize):
        for j in range(boardSize):
            for k in range(boardSize):
                decisionVars[i,j,k] = cp.Variable(boolean=True, name=f"x_{i}{j}{k}")

    constraints = []
    for j in range(boardSize):
        for k in range(boardSize):
            constraints += [np.sum(decisionVars[:,j,k]) == 1]

    constraints = []
    for i in range(boardSize):
        for k in range(boardSize):
            constraints += [np.sum(decisionVars[i,:,k]) == 1]

    objective = cp.Maximize( profitTable[:k].T @ cuts  )

    lp = cp.Problem(objective, constraints)
    lp.solve(solver="MOSEK", verbose=False)

    optimum = cuts.value.round(1)
    optimum = optimum.tolist()
    optimalValue = int(lp.value)

    return optimum, optimalValue
    return solution

NameError: name 'np' is not defined

## Questão 3: Senha

## Questão 4: 8 rainhas